This is a notebook showing how the final data used for data analysis and hypothesis testing is build from all the available data files stored in eye_data and input_data respectively.

In [1]:
import os
import math
import numpy as np
import pandas as pd
from ast import literal_eval
import itertools
import arviz as az
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as  mpatches
pd.options.mode.chained_assignment = None  # default='warn'

from helper_functions import *

agent_center_x = 972
agent_center_y = 288

In [9]:
example = pre_process_eye_data(pd.read_csv(("../../../../Volumes/KMODyS_1/Data Moonlander III/AU25IL/eye_data/AU25IL_eye_tracking_output_1_0.0_15.csv"), index_col=False))
example

,time_tag,TimeTag,LeftEyeX,LeftEyeY,LeftPupilDiameter,RightEyeX,RightEyeY,RightPupilDiameter,DigitalIn,LeftBlink,...,distance_to_spaceship_in_pixel,distance_to_spaceship,distant_fixation,Saccade,saccadeOnset,N_saccade,saccade_direction_x,saccade_direction_y,saccade_amplitude_in_pixel,saccade_amplitude
0,0.0000,1540.3095,NaN,NaN,9999.0,-496.542694,-278.716827,80.042969,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,0.0005,1540.3100,NaN,NaN,9999.0,-489.150726,-278.826385,79.917969,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
2,0.0010,1540.3105,NaN,NaN,9999.0,-485.297089,-278.973694,79.933594,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
3,0.0015,1540.3110,NaN,NaN,9999.0,-481.454651,-279.178192,79.949219,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
4,0.0020,1540.3115,NaN,NaN,9999.0,-478.227936,-279.218628,79.988281,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87011,43.5055,1583.8150,NaN,NaN,9999.0,NaN,NaN,9999.000000,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
87012,43.5060,1583.8155,NaN,NaN,9999.0,NaN,NaN,9999.000000,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
87013,43.5065,1583.8160,NaN,NaN,9999.0,NaN,NaN,9999.000000,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN
87014,43.5070,1583.8165,NaN,NaN,9999.0,NaN,NaN,9999.000000,16777215.0,1.0,...,NaN,NaN,0,0.0,0,NaN,NaN,NaN,NaN,NaN


In [26]:
#root_dir = os.getcwd()

data_dir = "../../../../Volumes/KMODyS_1/Data Moonlander III/"

target_string = "output"
target_string_eye_tracking = "eye_tracking"

runs = []

for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        if target_string_eye_tracking in file:
            runs.append(file)

# all data files have in their names a triplet for identification...       
# triplet coding: level, drift (True vs. False), input noise (None vs. Weak vs. Strong)

In [27]:
runs.remove('._AU25IL_eye_tracking_output_3_1.5_22.csv')  # don't know what that is
runs

['AU25IL_eye_tracking_output_1_0.0_15.csv',
 'AU25IL_eye_tracking_output_1_0.5_31.csv',
 'AU25IL_eye_tracking_output_1_1.0_44.csv',
 'AU25IL_eye_tracking_output_1_1.5_34.csv',
 'AU25IL_eye_tracking_output_1_2.0_27.csv',
 'AU25IL_eye_tracking_output_1_2.0_38.csv',
 'AU25IL_eye_tracking_output_2_0.0_13.csv',
 'AU25IL_eye_tracking_output_2_0.5_04.csv',
 'AU25IL_eye_tracking_output_2_1.0_18.csv',
 'AU25IL_eye_tracking_output_2_1.5_08.csv',
 'AU25IL_eye_tracking_output_2_2.0_12.csv',
 'AU25IL_eye_tracking_output_2_2.0_14.csv',
 'AU25IL_eye_tracking_output_2_2.0_19.csv',
 'AU25IL_eye_tracking_output_3_0.0_29.csv',
 'AU25IL_eye_tracking_output_3_0.5_24.csv',
 'AU25IL_eye_tracking_output_3_1.0_20.csv',
 'AU25IL_eye_tracking_output_3_1.5_16.csv',
 'AU25IL_eye_tracking_output_3_1.5_22.csv',
 'AU25IL_eye_tracking_output_3_2.0_03.csv',
 'AU25IL_eye_tracking_output_3_2.0_07.csv',
 'AU25IL_eye_tracking_output_3_2.0_33.csv',
 'AU25IL_eye_tracking_output_4_0.0_25.csv',
 'AU25IL_eye_tracking_output_4_0

### finding dubious data

one trial of ID IT02AH shouldn't have any input. It is a trial in which the monitor malfunctioned. We're going to exclude that one.

In [58]:
data_dir = "../../../../Volumes/KMODyS_1/Data Moonlander III/IT02AH/data/"

for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        tmp_data = pre_process_input_data(pd.read_csv((f"{data_dir}{file}"), index_col=False))
                
        if 1 not in np.unique(tmp_data.start_input):
            print(file)


IT02AH_output_3_2.0_00.csv


In [60]:
runs.remove('IT02AH_eye_tracking_output_3_2.0_00.csv')
runs

['AU25IL_eye_tracking_output_1_0.0_15.csv',
 'AU25IL_eye_tracking_output_1_0.5_31.csv',
 'AU25IL_eye_tracking_output_1_1.0_44.csv',
 'AU25IL_eye_tracking_output_1_1.5_34.csv',
 'AU25IL_eye_tracking_output_1_2.0_27.csv',
 'AU25IL_eye_tracking_output_1_2.0_38.csv',
 'AU25IL_eye_tracking_output_2_0.0_13.csv',
 'AU25IL_eye_tracking_output_2_0.5_04.csv',
 'AU25IL_eye_tracking_output_2_1.0_18.csv',
 'AU25IL_eye_tracking_output_2_1.5_08.csv',
 'AU25IL_eye_tracking_output_2_2.0_12.csv',
 'AU25IL_eye_tracking_output_2_2.0_14.csv',
 'AU25IL_eye_tracking_output_2_2.0_19.csv',
 'AU25IL_eye_tracking_output_3_0.0_29.csv',
 'AU25IL_eye_tracking_output_3_0.5_24.csv',
 'AU25IL_eye_tracking_output_3_1.0_20.csv',
 'AU25IL_eye_tracking_output_3_1.5_16.csv',
 'AU25IL_eye_tracking_output_3_1.5_22.csv',
 'AU25IL_eye_tracking_output_3_2.0_03.csv',
 'AU25IL_eye_tracking_output_3_2.0_07.csv',
 'AU25IL_eye_tracking_output_3_2.0_33.csv',
 'AU25IL_eye_tracking_output_4_0.0_25.csv',
 'AU25IL_eye_tracking_output_4_0

## Building eye-movement dataframe

In [77]:
# filter the content for trials in snippets df
eye_data_complete = pd.DataFrame()

# slow but explicit for-loop...
for run in runs:
    
    tmp = run.split('_')
    id_code = tmp[0]
    level = int(tmp[4])
    input_noise = tmp[5]
    exp_trial = tmp[-1][:2]
    
    tmp_data = pre_process_eye_data(pd.read_csv((f"../../../../Volumes/KMODyS_1/Data Moonlander III/{id_code}/eye_data/{run}"), index_col=False))

    tmp_data["ID"] = id_code
    tmp_data["level"] = level
    tmp_data["input_noise"] = input_noise
    tmp_data["trial"] = exp_trial

    eye_data_complete = pd.concat([eye_data_complete, tmp_data])

18646221 rows of data generated for final data set.

In [78]:
def func(x):
    if x == 1:
        return "medium"
    elif x == 2:
        return "medium"
    elif x == 3:
        return "medium"
    elif x == 4:
        return "hard"
    elif x == 5:
        return "hard"
    elif x == 6:
        return "hard"
    else:
        return 'N/A'

#eye_data_complete["level_difficulty"] = eye_data_complete.level.apply(func)

# Calling func for the sub data frames for fixations and saccades individually is more efficient (less time consuming).

### Extracting fixations

In [79]:
fix_data = eye_data_complete[eye_data_complete.fixationOnset == 1].reset_index()
len(fix_data)

31522

In [80]:
N_visible_obs_col = []
Dist_to_closest_obstacle_col = []

for index, row in fix_data.iterrows():
    
    input_file_name = f'{row.ID}_output_{row.level}_{str(row.input_noise)}_{row.trial}.csv'
    
    path = f'../../../../Volumes/KMODyS_1/Data Moonlander III/{id_code}/data/'
    
    file_found = False
    
    for subdir, dirs, files in os.walk(path):
        for data_file in files:
            if input_file_name in str(data_file):
                
                file_found = True
                
                #print(f'searching for {data_file}')
                
                input_data = pre_process_input_data(pd.read_csv(f'{path}/{data_file}', index_col=False))
                
                closest_frame = input_data.iloc[(input_data['time_played'] - row.time_tag).abs().argsort()[0]]
                
                distances = []
                for obstacle in closest_frame.visible_obstacles:
                    dist_squared = np.power(np.abs(obstacle[0] - agent_center_x), 2) + np.power(np.abs(obstacle[1] - agent_center_y), 2)
                    distances.append(np.sqrt(dist_squared))
                
                if len(distances) > 0:
                    Dist_to_closest_obstacle = np.min(distances)
                elif len(distances) == 0:
                    Dist_to_closest_obstacle = np.nan
                    
                N_visible_obstacles = len(closest_frame.visible_obstacles)
                
                N_visible_obs_col.append(N_visible_obstacles)
                Dist_to_closest_obstacle_col.append(Dist_to_closest_obstacle)
    
    if not file_found:
        N_visible_obs_col.append(np.nan)
        Dist_to_closest_obstacle_col.append(np.nan)

In [81]:
fix_data.insert (41, 'N_visible_obstacles', N_visible_obs_col)
fix_data.insert (41, 'Dist_to_closest_obstacles_in_pixel', Dist_to_closest_obstacle_col)

fix_data['Dist_to_closest_obstacles'] = fix_data.apply(lambda row: pixel_to_degree(row.Dist_to_closest_obstacles_in_pixel), axis=1)

In [82]:
fix_data = fix_data.astype({'level':'int'})

fix_data["level_difficulty"] = fix_data.level.apply(func)

In [83]:
# drop unnecessary columns for fixation data
fix_data = fix_data.drop(labels=["saccade_amplitude", "saccade_amplitude_in_pixel", "saccade_direction_y", "saccade_direction_x", "saccadeOnset", "N_saccade", "Saccade"], axis = 1)

fix_data.to_csv('fixations_moonlander_iii.csv', sep=',')

## Building SoC data

In [10]:
root_dir = os.getcwd()

data_dir = "/experimental_data/"
target_string = "output"

runs = []

for subdir, dirs, files in os.walk(root_dir+data_dir):
    for file in files:
        # potentially only going for crashed or done data
        if done_string in file:
            runs.append(file)
        if crashed_string in file:
            runs.append(file)

In [11]:
snippets_runs = []

for file_name in runs:
    temp = file_name.split("_")
    code = temp[0]
    exp_trial = temp[-1]
    exp_trial = exp_trial[:2]  # cut off .csv
    
    snippets_runs.append([code, exp_trial])

In [12]:
snippets_runs_df = pd.DataFrame(snippets_runs, columns = ['code', 'trial'])
snippets_runs_df

,code,trial
0,OK01UE,10
1,OK01UE,25
2,OK01UE,16
3,OK01UE,17
4,OK01UE,08
...,...,...
1465,UD06AD,35
1466,UD06AD,41
1467,UD06AD,42
1468,UD06AD,45


In [13]:
#%%capture --no-stderr

soc_data = pd.DataFrame()


for id_code in np.unique(snippets_runs_df.code):
    experimental_runs = list(snippets_runs_df.loc[snippets_runs_df['code'] == id_code].trial)
    
    path = root_dir + data_dir + str(id_code) + "/data"
    for subdir, dirs, files in os.walk(path):
        for data_file in files:
            if ".csv" in str(data_file):
                
                temp = str(data_file).split("_")
                # extract features of run from file_name coding
                level = temp[2][0]
                drift = temp[2][1]
                input_noise = temp[2][2]
                exp_trial = temp[-1]
                exp_trial = exp_trial[:2]
                done = temp[3]
                if int(level) > 0:  # excluding training trials
                
                    # writing to a temporal data frame
                    temp_data = pd.DataFrame({'ID': [np.nan],
                                              'level': [np.nan],
                                              'drift': [np.nan],
                                              'input_noise': [np.nan],
                                              'N_fixations': [np.nan],
                                              'N_distant_fixations': [np.nan],
                                              'N_saccades': [np.nan],
                                              'done': [np.nan],
                                              'run': [np.nan], 
                                              'time_played': [np.nan],
                                              'SoC': [np.nan]})

                    opened_file = pre_process_input_data(pd.read_csv((f"experimental_data/{id_code}/data/{data_file}"), index_col=False))
                    temp_eye_data_file = pre_process_eye_data(pd.read_csv(f"experimental_data/{id_code}/eye_data/{id_code}_eye_tracking_output_{level}{drift}{input_noise}_{exp_trial}.csv"))
                    temp_data["ID"].iloc[0] = id_code
                    temp_data["level"].iloc[0] = level
                    temp_data["drift"].iloc[0] = drift
                    temp_data["input_noise"].iloc[0] = input_noise
                    temp_data["N_fixations"].iloc[0] = np.max(temp_eye_data_file.N_fixation)
                    temp_data["N_distant_fixations"].iloc[0] = np.sum(temp_eye_data_file.exploring_fixation)
                    temp_data["N_saccades"].iloc[0] = np.max(temp_eye_data_file.N_saccade)                                                     
                    if done == "crashed":
                        temp_data["done"].iloc[0] = False
                    elif done == "done":
                        temp_data["done"].iloc[0] = True
                    temp_data["run"].iloc[0] = exp_trial
                    temp_data["time_played"].iloc[0] = opened_file["time_played"].iloc[-1]
                    temp_data["SoC"].iloc[0] = opened_file["SoC"].iloc[-1]

                    soc_data = pd.concat([soc_data, temp_data])

soc_data


,ID,level,drift,input_noise,N_fixations,N_distant_fixations,N_saccades,done,run,time_played,SoC
0,AE07EM1,3,T,N,64.0,43.0,29.0,True,09,29.145475,6.0
0,AE07EM1,2,F,S,128.0,83.0,75.0,True,43,59.888274,6.0
0,AE07EM1,4,T,W,133.0,86.0,46.0,True,46,60.120678,6.0
0,AE07EM1,5,F,W,59.0,36.0,26.0,True,10,28.551333,6.0
0,AE07EM1,6,T,W,119.0,56.0,37.0,False,12,40.899765,4.0
...,...,...,...,...,...,...,...,...,...,...,...
0,ZC12VE,4,T,W,70.0,50.0,43.0,False,30,40.508014,3.0
0,ZC12VE,4,F,S,119.0,80.0,57.0,True,24,58.614674,7.0
0,ZC12VE,4,T,S,109.0,92.0,72.0,True,54,58.915442,6.0
0,ZC12VE,5,T,W,17.0,11.0,11.0,False,47,9.739006,1.0


### Played time on average (reported in paper)

In [30]:
done_data = soc_data.loc[soc_data['done'] == True]

short_levels = done_data.loc[done_data['level'].isin(['1','3', '5'])]
long_levels = done_data.loc[done_data['level'].isin(['2','4', '6'])]

mean_short, sd_short = np.mean(short_levels.time_played), np.std(short_levels.time_played)
mean_long, sd_long = np.mean(long_levels.time_played), np.std(long_levels.time_played)

In [22]:
short_levels = soc_data.loc[soc_data['level'].isin(['1','3', '5'])]
long_levels = soc_data.loc[soc_data['level'].isin(['2','4', '6'])]

mean_short, sd_short = np.mean(short_levels.time_played), np.std(short_levels.time_played)
mean_long, sd_long = np.mean(long_levels.time_played), np.std(long_levels.time_played)

In [31]:
mean_short, sd_short, mean_long, sd_long

(29.08436093374433, 2.7096044266466603, 58.58936573927265, 2.0720307351422127)

In [32]:
# for levels individually:
for i in ['1', '2', '3', '4', '5', '6']:

    level_data = done_data.loc[done_data['level'] == i]

    mean, sd = np.mean(level_data.time_played), np.std(level_data.time_played)
    
    print(mean, sd)

28.927957169803573 1.211527961164191
58.627408046781284 2.2682027218477656
29.121662544615475 1.4652311261471465
58.592955037251414 1.9210791021262388
29.26137514289366 4.870696466385037
58.513255975463174 1.9333950821077057


In [40]:
mean, sd = np.mean(soc_data.groupby('ID')['time_played'].sum()), np.std(soc_data.groupby('ID')['time_played'].sum())
minimum, maximum = np.min(soc_data.groupby('ID')['time_played'].sum()), np.max(soc_data.groupby('ID')['time_played'].sum())
print(mean, sd, minimum, maximum)

1867.0823626165036 120.98819139879132 1572.4881093502045 2182.5611712932587


### Adding additional columns

In [ ]:
soc_df = pd.DataFrame()

unique_IDs = soc_data.ID.unique()

for ident in unique_IDs:
    # simultaneously sorting data by run (cronologically)
    temp_df = soc_data[soc_data.ID == ident].sort_values(['run'], ascending=True)
    
    # condition for crash in data
    cond = (temp_df.done == False)

    # have =1 everywhere condition applies and =0 where not
    temp_df["crashed"] = np.where(cond, 1, 0)
    
    # counting up crashs
    temp_df['N_prior_crashs'] = temp_df.crashed.cumsum()
    
    # we will have to subtract 1 from each row where they crashed due to cumsum already updating the crash row. 
    # But we want specifically the PRIOR crashs.
    temp_df.N_prior_crashs = temp_df.N_prior_crashs - temp_df.crashed
    
    soc_df = pd.concat([soc_df, temp_df])
    
soc_df

In [ ]:

soc_df_=soc_df.assign(trials_since_last_crash=soc_df.groupby(soc_df.crashed.ne(soc_df.crashed.shift()).cumsum()).cumcount().add(1))
soc_df_.trials_since_last_crash = soc_df_.trials_since_last_crash - soc_df_.crashed

#test_df = new_soc_df.copy()

cond = (soc_df_.crashed.shift(1) > 0.0)
soc_df_["crashed_in_last_trial"] = np.where(cond, 1, 0)

cond = (soc_df_.crashed.shift(1) > 0.0) & (soc_df_.crashed == 0)
soc_df_["consecutive_crash_success"] = np.where(cond, 1, 0)

soc_df_["N_consecutive_crash_success"] = soc_df_.groupby('ID')["consecutive_crash_success"].cumsum()

soc_df_


In [ ]:
soc_df_.to_csv('soc_data/soc_data.csv', sep=',', index=False)

## Building input data